<a href="https://colab.research.google.com/github/SHASHWATSHARMA2004/University-Query-Priority-Classification/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
train_data ="https://raw.githubusercontent.com/SHASHWATSHARMA2004/University-Query-Priority-Classification/refs/heads/main/university_query_train.csv"
test_data ="https://raw.githubusercontent.com/SHASHWATSHARMA2004/University-Query-Priority-Classification/refs/heads/main/university_query_test.csv"

In [9]:
import pandas as pd


In [12]:
train_df=pd.read_csv(train_data,usecols=['Student_Query', 'Department', 'Days_To_Deadline',
       'Priority_Label'])
test_df=pd.read_csv(test_data,usecols=['Student_Query', 'Department', 'Days_To_Deadline',
       'Priority_Label'])

In [11]:
train.sample()

,Query_ID,Student_Query,Department,Days_To_Deadline,Priority_Label
156,1039,I need a bonafide certificate.,Examination Cell,58,Medium


In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Query_ID          5000 non-null   int64 
 1   Student_Query     5000 non-null   object
 2   Department        5000 non-null   object
 3   Days_To_Deadline  5000 non-null   int64 
 4   Priority_Label    5000 non-null   object
dtypes: int64(2), object(3)
memory usage: 195.4+ KB


In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Query_ID          1000 non-null   int64 
 1   Student_Query     1000 non-null   object
 2   Department        1000 non-null   object
 3   Days_To_Deadline  1000 non-null   int64 
 4   Priority_Label    1000 non-null   object
dtypes: int64(2), object(3)
memory usage: 39.2+ KB


In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model

In [16]:
# find max len of text in data
max_len = train_df["Student_Query"].apply(lambda x: len(x.split())).max()
max_len

12

In [17]:
max_words = 10000
max_len = 15

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df["Student_Query"])

X_train_text = tokenizer.texts_to_sequences(train_df["Student_Query"])
X_test_text = tokenizer.texts_to_sequences(test_df["Student_Query"])

X_train_text = pad_sequences(X_train_text, maxlen=max_len, padding="post")
X_test_text = pad_sequences(X_test_text, maxlen=max_len, padding="post")

In [18]:
X_train_text

array([[13,  8, 55, ...,  0,  0,  0],
       [ 3, 74, 75, ...,  0,  0,  0],
       [13,  8, 30, ...,  0,  0,  0],
       ...,
       [12, 32, 11, ...,  0,  0,  0],
       [13,  8, 30, ...,  0,  0,  0],
       [19, 20, 21, ...,  0,  0,  0]], dtype=int32)

In [19]:
dept_encoder = LabelEncoder()

train_df["Department"] = dept_encoder.fit_transform(train_df["Department"])
test_df["Department"] = dept_encoder.transform(test_df["Department"])

num_departments = len(dept_encoder.classes_)

In [20]:
scaler = StandardScaler()

X_train_days = scaler.fit_transform(train_df[["Days_To_Deadline"]])
X_test_days = scaler.transform(test_df[["Days_To_Deadline"]])

In [21]:
label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(train_df["Priority_Label"])
y_test = label_encoder.transform(test_df["Priority_Label"])

num_classes = len(label_encoder.classes_)

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [22]:
text_input = Input(shape=(max_len,), name="text_input")

embedding = Embedding(input_dim=max_words,
                      output_dim=128,
                      input_length=max_len)(text_input)

lstm_out = LSTM(64)(embedding)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [23]:
dept_input = Input(shape=(1,), name="dept_input")

dept_embedding = Embedding(input_dim=num_departments,
                           output_dim=8)(dept_input)

dept_flat = tf.keras.layers.Flatten()(dept_embedding)

In [24]:
days_input = Input(shape=(1,), name="days_input")

In [25]:
concat = Concatenate()([lstm_out, dept_flat, days_input])

dense = Dense(64, activation="relu")(concat)
drop = Dropout(0.3)(dense)

output = Dense(num_classes, activation="softmax")(drop)

In [26]:
model = Model(
    inputs=[text_input, dept_input, days_input],
    outputs=output
)

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text_input          │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dept_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 15, 128)   │  1,280,000 │ text_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 1, 8)      │         56 │ dept_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 64)        │     49,408 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 8)         │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ days_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 73)        │          0 │ lstm[0][0],       │
│ (Concatenate)       │                   │            │ flatten[0][0],    │
│                     │                   │            │ days_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      4,736 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 3)         │        195 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,334,395 (5.09 MB)

 Trainable params: 1,334,395 (5.09 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
history = model.fit(
    [X_train_text, train_df["Department"], X_train_days],
    y_train,
    validation_data=(
        [X_test_text, test_df["Department"], X_test_days],
        y_test
    ),
    epochs=10,
    batch_size=32
)

Epoch 1/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.7338 - loss: 0.5142 - val_accuracy: 1.0000 - val_loss: 1.2779e-04
Epoch 2/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 1.0000 - loss: 5.5237e-04 - val_accuracy: 1.0000 - val_loss: 3.2537e-05
Epoch 3/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 1.0000 - loss: 1.5250e-04 - val_accuracy: 1.0000 - val_loss: 1.3093e-05
Epoch 4/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 1.0000 - loss: 9.8074e-05 - val_accuracy: 1.0000 - val_loss: 6.5696e-06
Epoch 5/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 1.0000 - loss: 6.7778e-05 - val_accuracy: 1.0000 - val_loss: 3.8270e-06
Epoch 6/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 1.0000 - loss: 5.1314e-05 - val_accuracy: 1.0000 - val_loss: 2.3478e-06
Epoch 7/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 1.0000 - loss: 4.2334e-05 - val_accuracy: 1.0000 - val_loss: 1.4806e-06
Epoch 8/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 14